# 02.4 — Pandas: Data Cleaning (Missing values, Duplicates, Types, Dropping Columns)

This notebook covers core data cleaning operations in Pandas:

- Detecting and handling missing values (`.isna()`, `.fillna()`, `.dropna()`)
- Finding and removing duplicates (`.duplicated()`, `.drop_duplicates()`)
- Type conversions (`.astype()`)
- Dropping columns (`.drop(columns=...)`)

Dataset used: **Titanic** (loaded from a stable raw GitHub URL). This notebook is Google Colab-ready.

---

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 60)

url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)
print('Loaded rows:', len(df))
df.head()

Loaded rows: 891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Missing Values

1. Check missing values per column using `.isna()` and `.sum()`.
2. Fill missing values with `.fillna()` — common strategies: median (numeric), mode, or a placeholder string for categorical.
3. Drop rows with missing values using `.dropna()` (use carefully).


In [3]:
# Check missing values
missing_counts = df.isna().sum()
print(missing_counts)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [4]:
# Show columns that have missing values
missing_counts[missing_counts>0]

,0
Age,177
Cabin,687
Embarked,2


In [5]:
# Fill numeric missing (Age) with median
age_median = df['Age'].median()
print('\nMedian Age:', age_median)
df['Age_filled'] = df['Age'].fillna(age_median)
print('Missing in Age_filled:', df['Age_filled'].isna().sum())


Median Age: 28.0
Missing in Age_filled: 0


In [6]:

# Fill Cabin with 'Unknown'
df['Cabin_filled'] = df['Cabin'].fillna('Unknown')
print('Missing in Cabin_filled:', df['Cabin_filled'].isna().sum())

# Example: drop rows where Embarked is missing (small number)
df_dropped = df.dropna(subset=['Embarked'])
print('\nRows after dropping missing Embarked:', len(df_dropped))

# Display head
df[['Age','Age_filled','Cabin','Cabin_filled','Embarked']].head()

Missing in Cabin_filled: 0

Rows after dropping missing Embarked: 889


,Age,Age_filled,Cabin,Cabin_filled,Embarked
0,22.0,22.0,NaN,Unknown,S
1,38.0,38.0,C85,C85,C
2,26.0,26.0,NaN,Unknown,S
3,35.0,35.0,C123,C123,S
4,35.0,35.0,NaN,Unknown,S


## Duplicates

- Use `.duplicated()` to mark duplicate rows.
- Use `.drop_duplicates()` to remove them. You can specify subset of columns to check duplicates on.


In [7]:
# Count duplicates
print('Total duplicates:', df.duplicated().sum())

Total duplicates: 0


In [11]:
# Create an artificial duplicate for demo (copy first row)
demo = df.copy()
# Use pd.concat instead of append
demo = pd.concat([demo, demo.iloc[[0]]], ignore_index=True)
print('After appending a duplicate, duplicates:', demo.duplicated().sum())

After appending a duplicate, duplicates: 1


In [13]:
# Drop duplicates (keep first)
demo_no_dup = demo.drop_duplicates()
print('After drop_duplicates, rows:', len(demo_no_dup))

# Drop duplicates based on subset of columns (Name and Fare)
df_no_dup_subset = df.drop_duplicates(subset=['Name','Fare'])
print('Rows after subset-based dedupe:', len(df_no_dup_subset))

After drop_duplicates, rows: 891
Rows after subset-based dedupe: 891


## Type Conversion

- Use `.astype()` to convert datatypes (e.g., float → int).
- Convert `Pclass` to a categorical type to save memory and express intent.


In [14]:
# Before conversion
print(df.dtypes[['Pclass','Fare']])

# Convert Fare to integer (demonstration) — careful: this truncates cents
df['Fare_int'] = df['Fare'].fillna(0).astype(int)

# Convert Pclass to category
df['Pclass_cat'] = df['Pclass'].astype('category')

print('\nAfter conversion')
print(df.dtypes[['Pclass','Pclass_cat','Fare','Fare_int']])

df[['Pclass','Pclass_cat','Fare','Fare_int']].head()

Pclass      int64
Fare      float64
dtype: object

After conversion
Pclass           int64
Pclass_cat    category
Fare           float64
Fare_int         int64
dtype: object


,Pclass,Pclass_cat,Fare,Fare_int
0,3,3,7.2500,7
1,1,1,71.2833,71
2,3,3,7.9250,7
3,1,1,53.1000,53
4,3,3,8.0500,8


## Dropping Columns

- Use `df.drop(columns=[...])` to remove unwanted columns.
- Use `inplace=True` to modify the DataFrame directly, or assign the result to a new variable.


In [15]:
# Demo: drop columns Ticket & Cabin (create a copy so original df remains)
df_copy = df.copy()
df_copy.drop(columns=['Ticket','Cabin'], inplace=True)
print('Columns after drop:', df_copy.columns.tolist())

df_copy.head()

Columns after drop: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Age_filled', 'Cabin_filled', 'Fare_int', 'Pclass_cat']


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Age_filled,Cabin_filled,Fare_int,Pclass_cat
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,22.0,Unknown,7,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,38.0,C85,71,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,26.0,Unknown,7,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,35.0,C123,53,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,35.0,Unknown,8,3


## Best Practices & Tips

- Always inspect `df.info()` to understand dtypes and missingness.
- Prefer filling missing values thoughtfully (median for skewed, mean otherwise).
- When converting types, be aware of precision loss (e.g., float → int).
- Use `subset` in `dropna()` / `drop_duplicates()` to avoid removing too much data.
- Keep original columns (or make copies) when you are experimenting.


## Exercises

1. Count missing values per column and visualize columns with missing entries.
2. Fill missing `Age` with median and create an `Age_group` column with bins: [0-12,13-19,20-35,36-60,60+].
3. Remove duplicate rows and report how many rows were removed.
4. Convert `Survived` to category and `Fare` to float (ensure no data loss).
5. Drop `Ticket` column and save the cleaned DataFrame to CSV: `02.4_titanic_cleaned.csv`.
